# Colocalization between placental mQTL, psychiatric and developmental traits

Idea: for all heritable CpGs, test for colocalization between traits and GWAS summary statistics for conditions where a significant proportion of heritability can be explained by placental mQTL.
Traits to include:
* Neuropsychiatric
    1. OCD
    1. TS
    1. ADHD
    1. ASD
    1. MDD
    1. BIP
    1. AN
    1. SCZ
* Developmental: 
    1. Pubertal Growth Start
    1. Asthma
    1. Child BMI
    1. Child Onset Asthma

In [2]:
library(tidyverse)
library(data.table)
library(coloc)
library(here)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.4     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

Warning message:
“package ‘tibble’ was built under R version 4.1.1”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose


here() starts at /scratch/st-dennisjk-1/wcasazza/sex_specific_mQTL



In [3]:
heritability <- fread(here("data","cis_heritability.txt.gz"))[Pval < 0.1]
if(file.exists(here("data","marginal_meta_maf01_heritable.txt.gz"))){
    marginal <- fread(here("data","marginal_meta_maf01_heritable.txt.gz"))
}else{
    marginal <- fread(here("data","delahaye_richs_marginal_meta_matrixeqtl_maf01.txt.gz"))[heritability,on=c(gene="probe"),nomatch=0]
    fwrite(marginal,here("data","marginal_meta_maf01_heritable.txt.gz"), sep='\t')
}
if(file.exists(here("data","male_meta_maf01_heritable.txt.gz"))){
    male <- fread(here("data","male_meta_maf01_heritable.txt.gz"))
}else{
    male <- fread(here("data","delahaye_richs_male_meta_matrixeqtl_maf01.txt.gz"))[heritability,on=c(gene="probe"),nomatch=0]
    fwrite(male,here("data","male_meta_maf01_heritable.txt.gz"), sep='\t')
}
if(file.exists(here("data","female_meta_maf01_heritable.txt.gz"))){
    female <- fread(here("data","female_meta_maf01_heritable.txt.gz"))
}else{
    female <- fread(here("data","delahaye_richs_female_meta_matrixeqtl_maf01.txt.gz"))[heritability,on=c(gene="probe"),nomatch=0]
    fwrite(female,here("data","female_meta_maf01_heritable.txt.gz"), sep='\t')
}